In [2]:
from helper import *
import warnings
warnings.filterwarnings('ignore')

import pandas as pd #a library to make the data more structured

In [4]:
fo = pd.ExcelFile('clean-data.xlsx') #read the data
df = pd.read_excel(fo, 'Sheet1') #create data frame

text = df['text']
text_list =  [i.split() for i in text]

In [6]:
print(len(text_list))

2390


In [5]:
print(text_list)

[['makna', 'maiyyah', 'surat', 'at', 'taubah'], ['tujuan', 'penciptaan', 'manusia'], ['kaidah', 'mengenal', 'syirik', 'kecil'], ['diberi', 'pahala', 'sesuai', 'kadar', 'niatnya'], ['pembagian', 'tauhid', 'al', 'quran'], ['kado', 'indah', 'hari', 'valentine'], ['empat', 'kaidah', 'penting', 'memahami', 'syirik', 'tauhid', 'terjemah', 'al', 'qawaaidul', 'arba'], ['iman', 'asma', 'wa', 'sifat', 'allah', 'keutamaannya'], ['makna', 'kedekatan', 'kebersamaan', 'allah'], ['mengucapkan', 'selamat', 'natal', 'basa', 'basi'], ['perayaan', 'natal', 'aqidah', 'al', 'wala', 'wal', 'al', 'bara', 'dianggap', 'usang'], ['perayaan', 'natal', 'aqidah', 'al', 'wala', 'wal', 'al', 'bara', 'dianggap', 'usang'], ['perayaan', 'natal', 'aqidah', 'al', 'wala', 'wal', 'al', 'bara', 'dianggap', 'usang'], ['perayaan', 'natal', 'aqidah', 'al', 'wala', 'wal', 'al', 'bara', 'dianggap', 'usang'], ['mengenal', 'syirik', 'ashghar', 'syirik', 'kecil'], ['allah', 'mendengar', 'dua', 'telinga'], ['ngalap', 'berkah', 'tubu

In [9]:
#Create Bigram & Trigram Models 
from gensim.models import Phrases
# Add bigrams and trigrams to docs,minimum count 10 means only that appear 10 times or more.
bigram = Phrases(text_list, min_count=10)
trigram = Phrases(bigram[text_list])

for idx in range(len(text_list)):
    for token in bigram[text_list[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            text_list[idx].append(token)
    for token in trigram[text_list[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            text_list[idx].append(token)

In [10]:
from gensim import corpora, models
# Create a dictionary representation of the documents.
dictionary = corpora.Dictionary(text_list)

dictionary.filter_extremes(no_below=5, no_above=0.2) 
#no_below (int, optional) – Keep tokens which are contained in at least no_below documents.
#no_above (float, optional) – Keep tokens which are contained in no more than no_above documents (fraction of total corpus size, not an absolute number).

print(dictionary)

Dictionary(495 unique tokens: ['at', 'makna', 'surat', 'manusia', 'kaidah']...)


In [11]:
#https://radimrehurek.com/gensim/tut1.html 
#build corpus

doc_term_matrix = [dictionary.doc2bow(doc) for doc in text_list]
#The function doc2bow converts document (a list of words) into the bag-of-words format

'''The function doc2bow() simply counts the number of occurrences of each distinct word, 
converts the word to its integer word id and returns the result as a sparse vector. 
The sparse vector [(0, 1), (1, 1)] therefore reads: in the document “Human computer interaction”, 
the words computer (id 0) and human (id 1) appear once; 
the other ten dictionary words appear (implicitly) zero times.'''

print(len(doc_term_matrix))
print(doc_term_matrix[100])

tfidf = models.TfidfModel(doc_term_matrix) #build TF-IDF model
corpus_tfidf = tfidf[doc_term_matrix]

2390
[(97, 1), (117, 1)]


In [12]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from numpy import array
#function to compute coherence values
def compute_coherence_values(dictionary, corpus, texts, limit, start, step):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, iterations=100)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        
    return model_list, coherence_values

In [13]:
start=1
limit=21
step=1
model_list, coherence_values = compute_coherence_values(dictionary, corpus=corpus_tfidf, 
                                                        texts=text_list, start=start, limit=limit, step=step)
#show graphs
import matplotlib.pyplot as plt
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

<Figure size 640x480 with 1 Axes>

In [14]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 6))

Num Topics = 1  has Coherence Value of 0.601214
Num Topics = 2  has Coherence Value of 0.576674
Num Topics = 3  has Coherence Value of 0.591684
Num Topics = 4  has Coherence Value of 0.605839
Num Topics = 5  has Coherence Value of 0.637291
Num Topics = 6  has Coherence Value of 0.624999
Num Topics = 7  has Coherence Value of 0.607144
Num Topics = 8  has Coherence Value of 0.601661
Num Topics = 9  has Coherence Value of 0.605501
Num Topics = 10  has Coherence Value of 0.630242
Num Topics = 11  has Coherence Value of 0.611022
Num Topics = 12  has Coherence Value of 0.611754
Num Topics = 13  has Coherence Value of 0.616038
Num Topics = 14  has Coherence Value of 0.626722
Num Topics = 15  has Coherence Value of 0.628877
Num Topics = 16  has Coherence Value of 0.602461
Num Topics = 17  has Coherence Value of 0.618047
Num Topics = 18  has Coherence Value of 0.627549
Num Topics = 19  has Coherence Value of 0.596357
Num Topics = 20  has Coherence Value of 0.610252


In [19]:
from pprint import pprint

model = LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=15)
pprint(model.print_topics())

[(0,
  '0.052*"seputar" + 0.050*"zakat" + 0.040*"sujud" + 0.035*"hukum" + '
  '0.032*"wa_sallam" + 0.032*"mengenal" + 0.025*"syaikh" + 0.023*"penting" + '
  '0.022*"al_quran" + 0.019*"rambut"'),
 (1,
  '0.190*"hukum" + 0.043*"pakaian" + 0.036*"tahun" + 0.028*"termasuk" + '
  '0.028*"perkara" + 0.028*"haram" + 0.028*"baru" + 0.021*"menuntut_ilmu" + '
  '0.021*"menuntut" + 0.020*"perempuan"'),
 (2,
  '0.056*"ilmu" + 0.050*"non_muslim" + 0.047*"hadits" + 0.038*"yahudi" + '
  '0.038*"fadhilah" + 0.035*"wanita" + 0.030*"fikih" + 0.025*"non" + '
  '0.023*"definisi" + 0.023*"amal"'),
 (3,
  '0.058*"kafir" + 0.054*"nabi" + 0.051*"orang_kafir" + 0.046*"ilmu" + '
  '0.039*"hadits" + 0.038*"orang" + 0.036*"makmum" + 0.030*"tetap" + '
  '0.022*"negeri" + 0.020*"air"'),
 (4,
  '0.047*"jamaah" + 0.042*"mengusap" + 0.040*"membaca_al" + 0.033*"keutamaan" '
  '+ 0.032*"jual_beli" + 0.032*"jual" + 0.031*"beli" + 0.023*"istri" + '
  '0.021*"membuat" + 0.019*"sifat"'),
 (5,
  '0.066*"tangan" + 0.052*"bida

In [20]:
model = LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=15)

for idx, topic in model.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.043*"zakat" + 0.038*"masjid" + 0.037*"tangan" + 0.034*"shalat" + 0.033*"fithri" + 0.028*"istri" + 0.027*"perayaan" + 0.024*"dekat" + 0.020*"hadits_palsu" + 0.020*"melihat"
Topic: 1 Word: 0.058*"keutamaan" + 0.041*"hari" + 0.039*"puasa" + 0.036*"hadits" + 0.033*"bulan" + 0.031*"al_quran" + 0.028*"tahun" + 0.027*"tiga" + 0.026*"perkara" + 0.026*"syubhat"
Topic: 2 Word: 0.048*"hadits_lemah" + 0.046*"makmum" + 0.045*"shalat_jumat" + 0.042*"shalat" + 0.034*"jumat" + 0.030*"imam" + 0.026*"tata_cara" + 0.026*"tata" + 0.026*"sikap" + 0.026*"pemerintah"
Topic: 3 Word: 0.071*"syariat" + 0.047*"golongan" + 0.041*"pakaian" + 0.041*"salah" + 0.040*"islam" + 0.040*"tetap" + 0.033*"pemimpin" + 0.019*"daging" + 0.018*"sunnah" + 0.017*"hukum"
Topic: 4 Word: 0.143*"hadits" + 0.055*"fatwa_ulama" + 0.040*"salaf" + 0.032*"hukum" + 0.029*"wa_sallam" + 0.028*"puasa" + 0.025*"fatwa" + 0.025*"syiah" + 0.024*"non_muslim" + 0.021*"sahabat"
Topic: 5 Word: 0.096*"hadits_dhaif" + 0.045*"dhaif" + 0.

In [17]:
import pandas as pd
top_words_per_topic = []
for t in range(model.num_topics):
    top_words_per_topic.extend([(t, ) + x for x in model.show_topic(t, topn = 10)])

#pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word', 'P']).to_csv("top_words.csv")
df = pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word','P']).to_csv("top_words2.csv")
print(df)

None


In [21]:
import gensim
import pyLDAvis.gensim;pyLDAvis.enable_notebook()

data = pyLDAvis.gensim.prepare(model, corpus_tfidf, dictionary)
print(data)
pyLDAvis.save_html(data, 'lda-gensim.html')

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
4     -0.120354  0.244994       1        1  8.005549
5      0.011069  0.009175       2        1  7.656130
6      0.168941  0.020651       3        1  7.098236
10     0.082247  0.069603       4        1  7.092897
8     -0.195622 -0.090252       5        1  7.050929
1     -0.071335  0.096441       6        1  6.996595
9      0.102704 -0.058699       7        1  6.516655
13     0.092308  0.030336       8        1  6.513836
11     0.054198  0.005527       9        1  6.402919
12    -0.117338 -0.090671      10        1  6.347825
14     0.050057 -0.107936      11        1  6.270071
7     -0.018401  0.054089      12        1  6.162909
0      0.049187 -0.010678      13        1  6.085779
2      0.003681 -0.075301      14        1  5.987722
3     -0.091343 -0.097277      15        1  5.811958, topic_info=     Category        Freq                            Ter

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud as wd

for t in range(model.num_topics):
    plt.figure(figsize=(7,6))
    plt.imshow(wd(max_font_size=50, min_font_size=6).fit_words(dict(model.show_topic(t, 200))))
    plt.axis("off")
    plt.title("Topic #" + str(t))
    plt.savefig("wcld-topic-#"+str(t)+".png", facecolor='k', bbox_inches='tight')

plt.show()